In [1]:
import matplotlib.pyplot as plt
from syncutils import *
import os

from tqdm import tqdm

In [2]:
base_path = 'sync_measure_data/dancers/'

dirs_to_visit = [base_path + 'video' + str(k+1) + '/' for k in range(4)]

file_names = []

for dir_path in dirs_to_visit:
    file_names.extend(map(lambda name: dir_path + name, os.listdir(dir_path)))

print(len(file_names))

477


In [3]:
# extract extrema-based features from `dancers`
#
# arguments:
#
#   dt        - time interval between 2 frames, in seconds
#   T_max     - maximum video duration, in seconds
#   n_dancers - maximum number of dancers to extract features from
#
def extract_features_from_dancers(dancers, dt = 0.1, T_max = 5, n_dancers = 3):

    N = math.ceil(T_max/dt) + 1    # number of `interpolated frames`

    # fixing the order of limb names to be lexicographical
    #
    limb_names = list(body.limb_nodes.keys())
    limb_names.sort()

    n_limbs = len(limb_names)  # currently 11


    # features[k] - features for frame k:
    #
    # [dancer[1..3]_limb[1..11]_extrema_presence, dancer[1..3]_limb[1..11]_extrema_value] : 
    #
    # list length: 2*n_dancers*n_limbs = 2*3*11 = 66
    #
    features = np.zeros((N, 2*n_dancers*n_limbs))


    for k in range(len(dancers)):

        if k >= n_dancers:
            break

        for limb_idx, limb_name in enumerate(limb_names):

            if limb_name not in dancers[k].limbs:
                continue

            f = dancers[k].limbs[limb_name]

            d2f = f.derivative().derivative()
            extrema = f.derivative().roots(extrapolate = False)

            dancer_idx = 2*n_limbs*k
            feature_idx = dancer_idx + 2*limb_idx

            for t in extrema:

                if t < T_max:

                    n = t/dt

                    frame_idx = int(n)
                    dn = n - int(n)

                    val = f(t)*np.sign(d2f(t))/math.pi

                    # extremum presence
                    features[frame_idx][feature_idx] += 1 - dn
                    features[frame_idx + 1][feature_idx] += dn

                    # extremum value
                    features[frame_idx][feature_idx + 1] += val
                    features[frame_idx + 1][feature_idx + 1] += val
            #end
        #end
    #end

    return features


In [5]:
# extract extrema-based features from `dancers`
#
# arguments:
#
#   dt        - time interval between 2 frames, in seconds
#   T_max     - maximum video duration, in seconds
#   n_dancers - maximum number of dancers to extract features from
#
def extract_features_from_dancers_v1(dancers, dt = 0.1, T_max = 5, n_dancers = 3):

    N = math.ceil(T_max/dt) + 1    # number of `interpolated frames`

    # fixing the order of limb names to be lexicographical
    #
    limb_names = list(body.limb_nodes.keys())
    limb_names.sort()

    n_limbs = len(limb_names)  # currently 11


    # features[k] - features for frame k:
    #
    # [dancer[1..3]_limb[1..11]_extrema_value] : 
    #
    # list length: n_dancers*n_limbs = 3*11 = 33
    #
    features = np.zeros((N, n_dancers*n_limbs))


    for k in range(len(dancers)):

        if k >= n_dancers:
            break

        for limb_idx, limb_name in enumerate(limb_names):

            if limb_name not in dancers[k].limbs:
                continue

            f = dancers[k].limbs[limb_name]

            d2f = f.derivative().derivative()
            extrema = f.derivative().roots(extrapolate = False)

            dancer_idx = n_limbs*k
            feature_idx = dancer_idx + limb_idx

            for t in extrema:

                if t < T_max:

                    n = t/dt

                    frame_idx = int(n)
                    dn = n - int(n)

                    val = d2f(t)/math.pi

                    # extremum value
                    features[frame_idx][feature_idx] += (1 - dn)*val
                    features[frame_idx + 1][feature_idx] += dn*val
            #end
        #end
    #end

    return features


In [4]:
# extracting and saving features

os.makedirs('sync_measure_data/features/', exist_ok = True)

for file_name in tqdm(file_names):

    dancers = load_dancers(file_name)
    features = extract_features_from_dancers(dancers, dt = 0.01)

    # 'sync_measure_data/dancers/video1/seg1_1a_2.json' -> 'video1_seg1_1a_2'
    #
    name = '_'.join(file_name.split('/')[-2:]).split('.')[0]

    np.savez('sync_measure_data/features/' + name + '.npz', features = features)


100%|████████████████████████████████████████████████████████████████████████████████| 477/477 [00:12<00:00, 37.93it/s]


In [6]:
# extracting and saving features

os.makedirs('sync_measure_data/features_v1/', exist_ok = True)

for file_name in tqdm(file_names):

    dancers = load_dancers(file_name)
    features = extract_features_from_dancers_v1(dancers, dt = 0.05)

    # 'sync_measure_data/dancers/video1/seg1_1a_2.json' -> 'video1_seg1_1a_2'
    #
    name = '_'.join(file_name.split('/')[-2:]).split('.')[0]

    np.savez('sync_measure_data/features_v1/' + name + '.npz', features = features)

100%|████████████████████████████████████████████████████████████████████████████████| 477/477 [00:29<00:00, 16.03it/s]
